In [2]:
from dotenv import load_dotenv;
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json

In [3]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [4]:
data = json.load(open("aseriods.json"))

In [5]:
processed_data = []
client = OpenAI()

In [6]:
for asteroid in data["near_earth_objects"]:

    response = client.embeddings.create(input=asteroid['name'], model="text-embedding-3-small")
    embedding = response.data[0].embedding
    
    processed_data.append({
        "values": embedding,
        "id": asteroid["id"],
        "metadata": {
            "name": asteroid["name"],
            "first_observation_date": asteroid["first_observation_date"],
            "last_observation_date": asteroid["last_observation_date"],
            "inclination": asteroid["inclination"],
            "equinox": asteroid["equinox"],
            "orbit_class": asteroid["orbit_class"]["orbit_class_description"],
            "close_approach_date": asteroid["upcoming_approach"]["close_approach_date"],
            "close_approach_time": asteroid["upcoming_approach"]["close_approach_time"],
            "relative_velocity_mph": asteroid["upcoming_approach"]["relative_velocity"]["miles_per_hour"],
            "miss_distance_miles": asteroid["upcoming_approach"]["miss_distance"]["miles"]
        }
    })



In [7]:
index = pc.Index("nasaneo")
upsert_respone = index.upsert(
    vectors=processed_data,
    namespace="asteroid",
)
